In [99]:
!pip install tensorflow

In [100]:
import tensorflow as tf
from pathlib import Path
from urllib.parse import urlparse
import os

In [103]:
import json
from pathlib import Path

def save_json(path: Path, data: dict):
    try:
        path.parent.mkdir(parents=True, exist_ok=True)
        
        with open(path, 'w') as json_file:
            json.dump(data, json_file, indent=4)
        print(f"JSON file saved successfully at: {path}")
    except Exception as e:
        print(f"Failed to save JSON file at {path}. Error: {e}")


In [117]:
class Evaluation:
    def __init__(self):
        self.path_to_model = "../artifacts/training/model.h5"
        self.training_data = os.path.join("../artifacts/data_ingestion","Chicken-fecal-images")
        self.params_image_size = [224,224,3]
        self.params_batch_size = 16

    def valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.params_image_size[:-1],
            batch_size = self.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(
            path
        )

    def initiate_evaluation(self):
        self.model = self.load_model(self.path_to_model)
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
            loss='categorical_crossentropy',  # or whatever loss you used during training
            metrics=[tf.keras.metrics.Accuracy()]
        )
        self.valid_generator()
        self.score = self.model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss":self.score[0], "accuracy":self.score[1]}
        save_json(path = Path("scores.json"), data = scores)
        print(scores)

In [115]:
if __name__ == "__main__":
    evaluation_obj = Evaluation()
    evaluation_obj.initiate_evaluation()
    evaluation_obj.save_score()

Found 20 images belonging to 2 classes.


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 444ms/step - accuracy: 0.2437 - loss: 0.3899
JSON file saved successfully at: scores.json
{'loss': 0.5036683082580566, 'accuracy': 0.22499999403953552}
